In [3]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils import get_db_connection, log


load_dotenv()

def load_data():
    with get_db_connection() as conn:
        esg = pd.read_sql("SELECT * FROM esg_events", conn)
        sat = pd.read_sql("SELECT * FROM satellite_signals", conn)
    return esg, sat

def get_company_sites():
    return pd.DataFrame([
        {"company": "Caterpillar", "site_id": "IL1", "lat": 40.0, "lon": -89.0, "region": "Illinois"},
        # Add more sites as needed
    ])

def build_features(esg, sat, company_sites):
    esg["date"] = pd.to_datetime(esg["date"])
    sat["acq_date"] = pd.to_datetime(sat["acq_date"])
    features = []

    for _, site in company_sites.iterrows():
        esg_site = esg[esg["location"].str.contains(site["region"], na=False)]
        sat_site = sat[
            (np.abs(sat["latitude"] - site["lat"]) <= 1.0) &
            (np.abs(sat["longitude"] - site["lon"]) <= 1.0)
        ]

        for day in pd.date_range("2025-01-01", "2025-06-30", freq="W"):
            esg_count = esg_site[(esg_site["date"] >= day - timedelta(days=7)) & (esg_site["date"] <= day)].shape[0]
            avg_frp = sat_site[(sat_site["acq_date"] >= day - timedelta(days=7)) & (sat_site["acq_date"] <= day)]["frp"].mean()

            features.append({
                "company": site["company"],
                "site_id": site["site_id"],
                "date": day,
                "esg_event_count_7d": esg_count,
                "avg_frp_7d": avg_frp if not np.isnan(avg_frp) else 0.0
            })

    return pd.DataFrame(features)

def save_features_to_db(df):
    db_url = os.getenv("DATABASE_URL").replace("postgresql+psycopg2", "postgresql")
    engine = create_engine(db_url)
    df.to_sql("risk_features", engine, index=False, if_exists="replace")
    log("✔ risk_features saved to PostgreSQL.")

if __name__ == "__main__":
    log("🚀 Starting feature engineering...")
    esg, sat = load_data()
    company_sites = get_company_sites()
    df = build_features(esg, sat, company_sites)
    save_features_to_db(df)


[LOG] 🚀 Starting feature engineering...
[LOG] ✔ risk_features saved to PostgreSQL.


/var/folders/3y/jcf5sjb1319blyc1b_cfn1xm0000gn/T/ipykernel_25178/3826413576.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  esg = pd.read_sql("SELECT * FROM esg_events", conn)
/var/folders/3y/jcf5sjb1319blyc1b_cfn1xm0000gn/T/ipykernel_25178/3826413576.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sat = pd.read_sql("SELECT * FROM satellite_signals", conn)
